# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [5]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time

In [6]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        
        userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        options = Options()
        options.add_argument(f"user-agent={userAgent}")
        options.add_argument("--headless")
        # use sellenium to get the page
        driver = webdriver.Chrome(options = options)#, executable_path= "C:/Users/r_sab/chromedriver.exe")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # Check if Title exists, add it to the Webpage class or otherwise ignore
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            # get rid of the irrelevant parts of the page
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            # get the remaining parts of the DOM and add them to the text
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        # get each "a" tag and if it has a href tag, add it to the links
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
ed = Website("https://edwarddonner.com")
ed.links

['#wp--skip-link--target',
 'https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwar

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#wp--skip-link--target
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwardd

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [13]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen3-235B-A22B',
 '/deepseek-ai/DeepSeek-Prover-V2-671B',
 '/nari-labs/Dia-1.6B',
 '/Qwen/Qwen3-30B-A3B',
 '/Qwen/Qwen3-32B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen3-Demo',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/stepfun-ai/Step1X-Edit',
 '/spaces/nvidia/describe-anything-model-demo',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/nvidia/Nemotron-CrossThink',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/rajpurkarlab/ReXGradient-160K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/di

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen3-235B-A22B
Updated
4 days ago
•
28.9k
•
660
deepseek-ai/DeepSeek-Prover-V2-671B
Updated
4 days ago
•
1.75k
•
633
nari-labs/Dia-1.6B
Updated
8 days ago
•
109k
•
1.79k
Qwen/Qwen3-30B-A3B
Updated
5 days ago
•
41.4k
•
433
Qwen/Qwen3-32B
Updated
6 days ago
•
75.7k
•
264
Browse 1M+ models
Spaces
Running
5.9k
5.9k
DeepSite
🐳
Generate any application with DeepSeek
Running
372
372
Qwen3 Demo
📊
Generate responses to user queries using conv

In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("Brainmustard", "http://brainmustard.com/")

Found links: {'links': [{'type': 'about page', 'url': 'http://brainmustard.com/About.html'}, {'type': 'cases page', 'url': 'http://brainmustard.com/Cases.html'}, {'type': 'articles page', 'url': 'http://brainmustard.com/Articles.html'}, {'type': 'contacts page', 'url': 'http://brainmustard.com/Contacts.html'}, {'type': 'services page', 'url': 'http://brainmustard.com/services.html'}, {'type': 'social media link', 'url': 'https://twitter.com/brain_mustard'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/5137247/'}]}


"You are looking at a company called: Brainmustard\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nBrain Mustard Inc.\nWebpage Contents:\n+1-416-556-3909\ncontact@brainmustard.com\nHome\nAbout\nSuccess Stories\nArticles\nContacts\nBrain\nMustard\nThe Missing Ingredient.\n.\nTECHNOLOGY\nMonitoring Ecosystems Using A.I. Enabled Interactive Maps\nBook a Demo Now\nTECHNOLOGY\nMonitoring Ecosystems Using A.I. Enabled Interactive Maps\nBook a Demo Now\nTECHNOLOGY\nMonitoring Ecosystems Using A.I. Enabled Interactive Maps\nBook a Demo Now\nOur Services\n1. Due Diligence for Influencer Marketing\nAny undesirable association with the brand ambassadors can pose severe threats to the brand equity of companies.\nUsing BrainMustard's technology, we conduct thorough and in-depth due diligence on all channels to mitigate the risks associated with brand ambassadors. This tech

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("Brainmustard", "http://brainmustard.com")

Found links: {'links': [{'type': 'about page', 'url': 'http://brainmustard.com/About.html'}, {'type': 'services page', 'url': 'http://brainmustard.com/services.html'}, {'type': 'contact page', 'url': 'http://brainmustard.com/Contacts.html'}, {'type': 'cases page', 'url': 'http://brainmustard.com/Cases.html'}, {'type': 'articles page', 'url': 'http://brainmustard.com/Articles.html'}, {'type': 'social media', 'url': 'https://twitter.com/brain_mustard'}, {'type': 'social media', 'url': 'https://www.linkedin.com/company/5137247/'}]}


```markdown
# Brain Mustard Inc. Brochure

## The Missing Ingredient in Technology Solutions

### About Us
At **Brain Mustard Inc.**, we believe that the right technology can transform brands and drive growth. Based in Toronto, Canada, we specialize in leveraging **Artificial Intelligence** to provide innovative solutions for monitoring ecosystems through interactive maps and more. Our mission is to empower businesses with cutting-edge tools that enhance their marketing strategies, improve customer experiences, and boost profitability.

### Our Services

1. **Due Diligence for Influencer Marketing**
   - We utilize advanced technology to conduct thorough due diligence on potential brand ambassadors, ensuring you align with the right individuals to maintain your brand equity.

2. **StreamInsights**
   - Our innovative solution identifies early-stage market opportunities and outliers, uncovering valuable consumer segments for targeted product development and marketing strategies.

3. **Interactive Customer Experience Mapping**
   - Through advanced AI algorithms, we create detailed maps capturing customer journey touchpoints. Our insights help businesses re-engineer customer experiences and shape lasting impressions.

4. **Geo-Targeted Influencer Marketing**
   - Post-COVID, businesses face tougher markets. We help clients access valuable local influencers with targeted marketing strategies that increase ROI and market share.

### Success Stories
- **BAUER:** Increased market share in a shrinking market.
- **SportChek:** Successfully repositioned the brand in consumer minds.
- **McKesson:** Established trust among consumers with strategic CX mapping.
- **Nikon:** Managed unwanted social influences effectively.

### Company Culture
At Brain Mustard, we foster a collaborative and innovative culture, where creativity and diverse perspectives are not just accepted, but celebrated. Our team is dedicated to continuous learning, staying ahead in technology trends, and providing stellar service to our clients. We believe that a supportive work environment is essential for driving successful outcomes for both our team and our clients.

### Careers at Brain Mustard
Are you passionate about technology and eager to contribute to a vibrant team? We're always looking for talented individuals who are committed to innovation and excellence. Join us and help shape the future of marketing technology!

### Contact Us
**Address:** 10 Alcorn Avenue, #204, Toronto, ON, M4V 3A9  
**Phone:** +1-416-556-3909  
**Email:** [Contact@BrainMustard.com](mailto:Contact@BrainMustard.com)  

---

### Book a Demo Today!
Explore the difference that **Brain Mustard's** advanced technology can make for your business. Let us show you how we can help you unlock new opportunities and transform your marketing strategy.
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or '' # read the chunk content
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

**The AI Community Building the Future**  
Welcome to Hugging Face, where the machine learning community collaborates to create innovative models, datasets, and applications. Our platform is designed to foster collaboration and streamline the development of AI technologies.

---

## Our Offerings
- **Models**: Access over **1 million+ models**, including state-of-the-art resources such as Transformers, Diffusers, and Tokenizers. 
- **Datasets**: Explore **250k+ datasets** available for various ML tasks to enhance your projects.
- **Spaces**: Discover and run applications, including DeepSeek and ICEdit, designed for various AI-related tasks.

### Key Services
- **Compute Solutions**: Affordable compute starting at **$0.60/hour for GPU**, enabling users to deploy applications efficiently.
- **Enterprise Solutions**: Tailored enterprise-grade security and support, starting at **$20/user/month**. 

> Join over **50,000 organizations** including Amazon, Google, and Microsoft who trust Hugging Face for their AI needs.

---

## Company Culture
At Hugging Face, we believe in **democratizing machine learning**. Our open-source philosophy encourages collaboration, innovation, and sharing knowledge across the global AI community. We prioritize a supportive environment where everyone’s contributions matter, ensuring that your work can directly impact the future of AI.

Our team comprises over **215 dedicated professionals** passionate about making machine learning easier and more accessible for everyone. 

---

## Careers at Hugging Face
We are always looking for talented individuals who share our vision of an inclusive AI community. Positions are available across various departments, including engineering, research, marketing, and community management. 

### Benefits of Joining Us:
- Work in a pioneering environment at the forefront of AI developments.
- Collaborate with a diverse group of innovators and experts.
- Flexible work-from-home and remote options.

---

Join us in building the future of AI!  
  
**Explore, Collaborate, and Innovate with Hugging Face**  
- **Website**: [huggingface.co](https://huggingface.co)  
- **Follow Us**:   
  - GitHub  
  - Twitter  
  - LinkedIn  
  - Discord  

### Let's build amazing AI solutions together!

In [27]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Brainustard", "http://brainmustard.com")

Found links: {'links': [{'type': 'about page', 'url': 'http://brainmustard.com/About.html'}, {'type': 'cases page', 'url': 'http://brainmustard.com/Cases.html'}, {'type': 'services page', 'url': 'http://brainmustard.com/services.html'}, {'type': 'contacts page', 'url': 'http://brainmustard.com/Contacts.html'}]}


# Brain Mustard Inc. Brochure

---

## About Us
Brain Mustard Inc. is a pioneering company that specializes in leveraging advanced technologies to enhance brand equity and market opportunities. We provide state-of-the-art solutions using AI-enabled interactive maps to monitor ecosystems and optimize marketing strategies. Our innovative approach allows businesses to navigate competitive landscapes more effectively.

### Mission
Our mission is to empower businesses with the insights and tools needed to thrive in a rapidly changing market environment.

---

## What We Offer

### 1. Due Diligence for Influencer Marketing
We conduct in-depth due diligence to safeguard brand equity from undesirable associations with brand ambassadors. Our technology helps clients mitigate risks while benefiting from long-term campaigns.

### 2. New Opportunities & Outliers with StreamInsights
Our exploratory solution helps clients identify early-stage market opportunities and outliers. By revealing insights about often-overlooked consumer segments, we assist companies in enhancing their product development and marketing strategies.

### 3. Experience Mapping
Using advanced AI algorithms, we capture numerous customer touchpoints and highlight key moments in the customer journey. This enables clients to re-engineer customer experiences optimally.

---

## Success Stories

- **Bauer**: Increased market share in a shrinking market.
- **SportChek**: Successfully repositioned the brand in the minds of consumers.
- **McKesson**: Built trust among consumers through targeted strategies.
- **Metrolinx**: Increased non-fare revenue with our customer experience mapping.
- **Nikon**: Managed social influences effectively in their ecosystem.
- **Spencer's**: Achieved record-breaking sales by managing social influence.
- **One Tea Leaf**: Identified profitable untapped market opportunities. 

---

## Company Culture
At Brain Mustard, we foster a culture of innovation, collaboration, and integrity. Our team is dedicated to pushing boundaries and constantly seeking new insights to help our clients succeed. We value creative thinking and encourage our employees to voice new ideas and strategies.

---

## Join Us
We are always on the lookout for talented individuals who are passionate about technology and marketing. If you are interested in being part of a forward-thinking team that values your contributions, explore our current job openings. 

### Contact Us
- **Address**: 10 Alcorn Avenue, #204, Toronto, ON, M4V 3A9
- **Phone**: +1-416-556-3909
- **Email**: [contact@brainmustard.com](mailto:contact@brainmustard.com)

---

## Connect With Us
Follow us on social media to stay updated on our latest solutions and industry insights!

---

**Brain Mustard Inc. - The Missing Ingredient for Your Brand Success.**

© 2025 Brain Mustard Inc.  |  Privacy Policy

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>